In [1]:
import pandas as pd

In [2]:
import json
import logging
import os
import sys
import boto3

from dotenv import load_dotenv
from loguru import logger
import numpy as np
import pandas as pd
import datetime as dt
import s3fs
import sqlalchemy
from sqlalchemy import text

import geopandas as gpd
from shapely.geometry import LineString

sys.path.append('..')

from src.preprocessing import data_preprocessing_nshift, data_preprocessing_bridge

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

project_path = 's3://sfgdata/projects/sustainable-transport'

ModuleNotFoundError: No module named 'boto3'

### Dictionary for column names

In [54]:
import boto3
import s3fs
import json

# here for oerebro dataset
project_path = 's3://sfgdata/projects/sustainable-transport'
dict_column_names = 'bridge_names'

client = boto3.client('location')
s3 = s3fs.S3FileSystem()


with s3.open(f'{project_path}/external/nshift_names.json', 'r') as fp:
    dict_names = json.load(fp)

In [55]:
dict_names

{'Pickup date': 'Pickup date',
 'Client no': 'Receiver reference',
 'DC name': 'Actor',
 'Client name': 'Receiver',
 'Address': 'Receiver address',
 'Zip': 'Receiver zip',
 'City': 'Receiver city',
 'Country code': 'Receiver country code',
 'Carrier name': 'Carrier',
 'Sender weight (kg)': 'Sender weight (kg)',
 'Volume (m3)': 'Sender volume (l)',
 'Nb of packages': 'Number of packages'}

In [52]:
dict_names = {'Pickup date': 'ABF',
'DC code': 'TPLST',
'DC name': 'TPLSTLib',
'Client no': 'KUNNZ',
'Client name': 'KUNNZLib',
'Address': 'Street',
'Zip': 'PostCode',
'City': 'City',
'Country code': 'Country',
'Shipping type': 'VSART',
'Carrier code': 'TDLNR',
'Carrier name': 'TDLNRLib',
'Sender weight': 'YYBTGEW',
'Volume': 'YYVOLUM',
'Weight unit': 'YYGEWEI',
'Volume unit': 'YYVOLEH',
'Nb of packages': 'NbColis',
'Shipment id': 'TKNUM',
'Shipping point': 'VSTEL'}

In [57]:
df.columns.to_list()

['IdSysteme',
 'TKNUM',
 'SHTYP',
 'SHTYPLib',
 'TPLST',
 'TPLSTLib',
 'VSART',
 'VSARTLib',
 'VSBED',
 'VSBEDLib',
 'ROUTE',
 'ABF',
 'TDLNR',
 'TDLNRLib',
 'TSNUM',
 'VSTEL',
 'VSTELLib',
 'KUNNZ',
 'YYEXIDV',
 'KUNNR',
 'KUNAG',
 'Sum_BTGEW',
 'Sum_NTGEW',
 'GEWEI',
 'Sum_VOLUM',
 'VOLEH',
 'POSNR',
 'PARVW',
 'KUNNR1',
 'PARNR',
 'ADRNR',
 'ABLAD',
 'LAND1',
 'NAME1',
 'STREET',
 'CITY1',
 'POST_CODE1',
 'REGION',
 'COUNTRY']

In [47]:
dict_bridge_v2 = {'Pickup date': 'ABF',
'DC code': 'TPLST',
'DC name': 'TPLSTLib',
'Client no': 'KUNNR',
'Client name': 'NAME1',
'Address': 'STREET',
'Zip': 'POST_CODE1',
'City': 'CITY1',
'Country code': 'COUNTRY',
'Shipping type': 'VSART',
'Carrier code': 'TDLNR',
'Carrier name': 'TDLNRLib',
'Sender weight': 'Sum_BTGEW',
'Volume': 'Sum_VOLUM',
'Weight unit': 'GEWEI',
'Volume unit': 'VOLEH',
'Shipment id': 'TKNUM',
'Shipping point code': 'VSTEL',
'Shipping point': 'VSTELLib'
}

In [53]:
with s3.open(f'{project_path}/external/bridge_names.json', 'w') as fp:
    json.dump(dict_names, fp)

In [48]:
with s3.open(f'{project_path}/external/bridge_names_v2.json', 'w') as fp:
    json.dump(dict_bridge_v2, fp)

## Data Import

Information about data:
* Data from SAP stored in SQL database
* Data from Nshift stored in S3 in the "raw" folder:

### SAP data in SQL database

Data from SAP stored in SQL database:
* POC1: filter_vsartlib = ('GRD groupage','GRD LTL','GRD FTL','Truck')
* POC2: filter_vsartlib = ('GRD groupage','GRD LTL','GRD FTL','Truck', 'Shuttle', 'GRD Mono Parcel')
* POC3: Merge of POC2 data and NShift data of Oerebro

In [ ]:
dataset_name = 'sql_poc4'
df = pd.read_csv(f'{project_path}/raw/{dataset_name}.csv')

In [ ]:
dataset_name = 'sql_poc4'
df = pd.read_csv(f'{project_path}/processed/{dataset_name}.csv')

In [ ]:
project_path = 's3://sfgdata/projects/sustainable-transport'
dataset_name = 'poc4'

df = pd.read_csv(f'{project_path}/processed/{dataset_name}.csv', 
                 low_memory=False,
                 parse_dates=['Pickup date'], 
                 dtype={'DC zip': str, 'Zip': str}).sort_values(by='Pickup date')
df = df.dropna(subset = ['Distance (km)'])

#### Optional: Loading data from SQL to S3

In [33]:
# Save data to s3
odbc_connect = (
    f"DRIVER={os.getenv('SQL_DRIVER')};" + 
    f"SERVER={os.getenv('SQL_SERVER')};" + 
    f"PORT=1433;" +
    f"DATABASE={os.getenv('SQL_DATABASE')};" +
    f"UID={os.getenv('USER_NAME')};" +
    f"PWD={os.getenv('PASSWORD')}"
)

engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % odbc_connect)

In [34]:
filter_tplst = ('HU07','FR45','FR43')
filter_vsartlib = ('GRD groupage','GRD LTL','GRD FTL','Truck','Shuttle','GRD Mono Parcel')
filter_country = ('BE','EL','LT','PT','BG','ES','LU','RO','CZ','FR','HU','SI','DK','HR','MT','SK','DE','IT','NL','FI','EE','CY','AT','SE','IE','LV','PL')
#filter_kunnz = 'ISNUMERIC([KUNNZ]) = 1'
query = f'SELECT * FROM .[Perf].[vwTRP_Transport] WHERE TPLST IN {filter_tplst} AND VSARTLib IN {filter_vsartlib} AND COUNTRY IN {filter_country}'

with engine.begin() as conn:
    query = text(query)
    df = pd.read_sql_query(query, conn)

In [36]:
df[(df['VSTEL']=='FR20')&(df['TPLST']=='FR43')&(df['TKNUM']=='CS01167806')]

,IdSysteme,TKNUM,SHTYP,SHTYPLib,TPLST,TPLSTLib,VSART,VSARTLib,VSBED,VSBEDLib,ROUTE,ABF,TDLNR,TDLNRLib,TSNUM,VSTEL,VSTELLib,KUNNZ,YYEXIDV,KUNNR,KUNAG,Sum_BTGEW,Sum_NTGEW,GEWEI,Sum_VOLUM,VOLEH,POSNR,PARVW,KUNNR1,PARNR,ADRNR,ABLAD,LAND1,NAME1,STREET,CITY1,POST_CODE1,REGION,COUNTRY
156288,2,CS01167806,Z2G2,IG-MultiLeg-Out,FR43,DC Newlog,49,GRD groupage,Z7,Stock Order,FR3960,2022-01-11 19:39:00,0010043514,GEBRUDER WEISS GMBH,1,FR20,FR45-Standard,CSE00030,00636064800913463010,CSE00030,CSE00010,9264.0,8387.0,G,0.09,M3,0,WE,CSE00030,0,0000348619,POINT1,SE,SCHNEIDER ELECTRIC DISTRIBUTION CENTREAB,LASTGATAN,OREBRO,702 27,None,SE


In [37]:
df.groupby(['TPLST', 'TPLSTLib', 'VSTEL', 'VSTELLib']).agg(occurence = ('VSTEL', 'count'))

occurence
TPLST TPLSTLib           VSTEL VSTELLib                               
FR43  DC Newlog          BG01  BG02 - Standard                    1003
                         FR20  FR45-Standard                       354
                         FRD1  FR43-Standard-canalis               263
                         FRD2  FR43- Express-Canalis                 2
                         FRD3  FR43-Taxi Colis-Canalis               1
                         FRD4  FR43-Export-Canalis                2563
                         FRI1  FRA8-Chasseneuil-Standard          4952
                         FRI5  FRA8-Chasseneuil-Pces                 4
                         FRM7  FR43- Standard                   192950
                         FRM9  FR43- Express                       690
                         FRMA  FR43- Taxi colis                     80
                         FRMB  FR43- Export                        685
                         HU05  HU07-Normal SP Zala Factory         253
FR45  DC Evreux          FR20  FR45-Standard                    371627
                         FR21  FR45-Parachevement                 1057
                         FR22  FR45-Express                        278
                         FR24  FR45-Export                           1
                         FR26  FR45-Taxi Colis                      24
                         FR28  FR45-VAS 1                         2466
HU07  HU08-TPP RDC Cent. HU05  HU07-Normal SP Zala Factory         162
                         HU09  HU08-Normal SP Budapest RDC      216173
                         HU10  HU08-Returns SP Budapest RDC         24
                         HU15  HU08-Express SP Budapest RDC         45
                         HU24  HU08-0 day PP Budapest RDC         2576

> Anomalies in VSTEL; need to be investigated

In [ ]:
f'{df.shape[0]:,.0f}'

In [38]:
df.columns

Index(['IdSysteme', 'TKNUM', 'SHTYP', 'SHTYPLib', 'TPLST', 'TPLSTLib', 'VSART',
       'VSARTLib', 'VSBED', 'VSBEDLib', 'ROUTE', 'ABF', 'TDLNR', 'TDLNRLib',
       'TSNUM', 'VSTEL', 'VSTELLib', 'KUNNZ', 'YYEXIDV', 'KUNNR', 'KUNAG',
       'Sum_BTGEW', 'Sum_NTGEW', 'GEWEI', 'Sum_VOLUM', 'VOLEH', 'POSNR',
       'PARVW', 'KUNNR1', 'PARNR', 'ADRNR', 'ABLAD', 'LAND1', 'NAME1',
       'STREET', 'CITY1', 'POST_CODE1', 'REGION', 'COUNTRY'],
      dtype='object')

In [39]:
# Saving raw data to s3
df.to_csv(f'{project_path}/raw/sql_v2.csv', index=False)

### Data from NShift

* Celonis Hackathon dataset:
* Oerebro 2022
* Newlog 2022
* Evreux 2022

In [ ]:
from s3fs import S3FileSystem
s3_fs = S3FileSystem()

In [ ]:
s3_fs.find('s3://sfgdata/projects/sustainable-transport/raw/')

In [ ]:
# Hackathon
#data_nshift = pd.read_csv('s3://sfgdata/projects/sustainable-transport/raw/SE_nShift_Oct22.csv', sep='|', decimal=",", thousands=".")

# Newlog and Oerebro
#data_nshift = pd.read_csv(f's3://sfgdata/projects/sustainable-transport/raw/Extract_nShift_Newlog_2022.csv')
#data_nshift = pd.read_csv(f's3://sfgdata/projects/sustainable-transport/raw/Extract_nShift_Örebro_2022.csv', delimiter = ';', decimal = '.')

# Evreux
data_nshift = pd.read_csv(f's3://sfgdata/projects/sustainable-transport/raw/Extract_nShift_Evreux_2022.csv')

# EDA

### To Do: Data Profiling

In [ ]:
profile = ProfileReport(data_nshift, title="Pandas Profiling Report")
profile.to_file("pandas_profiling_report.html")

### Manual exploration: SQL

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Unique clients
df[['Street', 'City','PostCode', 'Country']].drop_duplicates()

In [ ]:
# Entries per DC
df.TPLSTLib.value_counts()

In [ ]:
df.columns

In [ ]:
print('Min Date: ', pd.to_datetime(
    df['ABF'], errors='coerce').dropna().min())
print('Max Date: ', pd.to_datetime(
    df['ABF'], errors='coerce').dropna().max())

In [ ]:
unique_shipper = df['TPLSTLib'].unique()
unique_receiver = df[['Street', 'PostCode', 'City', 'Country']].drop_duplicates().reset_index(drop=True).shape[0]
unique_routes = df[['TPLSTLib', 'Street', 'PostCode', 'City', 'Country']].drop_duplicates().reset_index(drop=True).shape[0]

# Change to upper or to lower
df_upper = df[['Street', 'City', 'PostCode', 'Country']]
for i in ['Street', 'City', 'Country']:
    df_upper[i] = df_upper[i].str.lower()
logger.info(f'Unique receiver  before cleaning: {unique_receiver:,.0f}')
logger.info(f'Unique receiver after cleaning: {df_upper.drop_duplicates().shape[0]:,.0f}')
logger.info(f'Unique shipper: {unique_shipper}')
logger.info(f'Unique routes: {unique_routes:,.0f}')

### Manual exploration: NShift

In [ ]:
data_nshift.info()

#### Unique routes

In [ ]:
unique_shipper = data_nshift[['Actor']].drop_duplicates().shape[0]

unique_receiver = data_nshift[['Receiver address', 'Receiver city', 'Receiver country code',
                               'Receiver zip']].drop_duplicates().shape[0]

unique_routes = data_nshift[['Actor', 'Receiver address', 'Receiver city', 'Receiver country code', 'Receiver zip']].drop_duplicates().shape[0]

print(f'Unique shipper: {unique_shipper}')
print(f'Unique receiver: {unique_receiver}')
print(f'Unique routes: {unique_routes}')

In [ ]:
data_nshift.Actor.unique()

In [ ]:
# Change to upper or to lower
df_upper = data_nshift[['Receiver country code', 'Receiver zip', 'Receiver city','Receiver address']]
for i in ['Receiver country code', 'Receiver city','Receiver address']:
    df_upper[i] = df_upper[i].str.lower()
print(f'Unique receiver before cleaning: {unique_receiver:,.0f}')
print(f'Unique receiver before cleaning: {df_upper.drop_duplicates().shape[0]:,.0f}')

#### Pickup date

In [ ]:
pd.to_datetime(data_nshift['Pickup date'], errors = 'coerce').min()

In [ ]:
pd.to_datetime(data_nshift['Pickup date'], errors = 'coerce').max()

In [ ]:
pd.to_datetime(data_nshift['Pickup date'], errors = 'coerce').isna().sum()

In [ ]:
data_nshift['Pickup date'] = pd.to_datetime(data_nshift['Pickup date'], errors = 'coerce')

### Possible Data Cleaning Steps

In [ ]:
# Missing values

In [ ]:
# Sender weight is Zero
df[df.YYBTGEW==0.0].head()

In [ ]:
# Change to upper or to lower
df_upper = df[['Street', 'City', 'PostCode', 'Country']]
for i in ['Street', 'City', 'Country']:
    df_upper[i] = df_upper[i].str.lower()
df_upper.drop_duplicates()

> Change to only upper/lower addresses: Less unique clients

### Data Cleaning

In [ ]:
dict_column_names = 'nshift_names'

client = boto3.client('location')
s3 = s3fs.S3FileSystem()
with s3.open(f'{project_path}/external/{dict_column_names}.json', 'r') as fp:
    dict_names_nshift = json.load(fp)

dict_names_nshift = {'Pickup date': 'Pickup date',
 'DC name': 'Actor',
 'Client name': 'Receiver',
 'Address': 'Receiver address',
 'Zip': 'Receiver zip',
 'City': 'Receiver city',
 'Country code': 'Receiver country code',
 'Carrier name': 'Carrier',
 'Sender weight (kg)': 'Sender weight (kg)',
 'Nb of packages': 'Number of packages'}

data_nshift = data_preprocessing_nshift(data_nshift, dict_names_nshift)
data_nshift['Pickup date'] = pd.to_datetime(data_nshift['Pickup date'])

In [ ]:
dict_column_names = 'bridge_names'

client = boto3.client('location')
s3 = s3fs.S3FileSystem()
with s3.open(f'{project_path}/external/{dict_column_names}.json', 'r') as fp:
    dict_names_bridge = json.load(fp)

df_sql = data_preprocessing_bridge(df, dict_names_bridge)
df_sql['Pickup date'] = pd.to_datetime(df_sql['Pickup date'])
#df_sql = df_sql[df_sql['DC name']=='DC Evreux']

In [ ]:
import re
valids = re.sub(r"[^A-Za-z]+", '', my_string)

In [ ]:
def check_string(x):
    if x.isalpha():
        return 1
    else:
        return 0

In [ ]:
import re

In [ ]:
def remove_hastag(x):
    x = re.sub(" # ", ' ', x)
    x = re.sub("# ", ' ', x)
    x = re.sub("#", ' ', x)
    x = re.sub(" , ", ' ', x)
    x = re.sub(", ", ' ', x)
    x = re.sub(",", ' ', x)
    x = re.sub(" - ", ' ', x)
    x = re.sub("- ", ' ', x)
    x = re.sub("-", ' ', x)
    return x

In [ ]:
df.Address.value_counts()[-50:]

In [ ]:
df.Address.iloc['GREMI FLORES, 31']

In [ ]:
remove_hastag('GREMI FLORES, 31')

In [ ]:
df.Address.apply(lambda x: remove_hastag(x))

In [ ]:
len(df.City.unique())

In [ ]:
df[df['City_string']==0].City.unique()

In [ ]:
df.KUNNZLib.unique().shape

In [ ]:
dict_names_bridge

### Data Validation: SQL vs NShift

In [ ]:
print(f"SQL: From {df_sql['Pickup date'].min()} to {df_sql['Pickup date'].max()}")
print(f"NShift: From {data_nshift['Pickup date'].min()} to {data_nshift['Pickup date'].max()}")

In [ ]:
df_nshift = data_nshift[(data_nshift['Pickup date']<="2022-12-25")&(data_nshift['Pickup date']>="2022-01-04")]
df_sql = df_sql[(df_sql['Pickup date']<="2022-12-25")&(df_sql['Pickup date']>="2022-01-04")]

In [ ]:
data_nshift['Sender weight (kg)'].sum()

In [ ]:
df_sql['Sender weight (kg)'].sum()

> Not the same weight

In [ ]:
# Change to upper or to lower
data_nshift[['Country code', 'Zip', 'City','Address']].drop_duplicates().shape[0]

In [ ]:
# Change to upper or to lower
df_sql[['Country code', 'Zip', 'City','Address']].drop_duplicates().shape[0]

> Not same number of unique receivers

In [ ]:
# In SQL: Shipment on 13-10
street = df_sql[(df_sql['Pickup date']>dt.datetime(2022,10,13))&(df_sql['Pickup date']<dt.datetime(2022,10,14))]['Address'].unique()[3]
df_sql[(df_sql['Address'] == street)&(df_sql['Pickup date']>dt.datetime(2022,10,13))&(df_sql['Pickup date']<dt.datetime(2022,10,14))][['Pickup date', 'DC name', 'Client name', 'Address', 'Zip', 'City','Country code', 'Carrier name', 'Sender weight (kg)']]

In [ ]:
# In NShift: Shipment on 13-10
df_nshift[(df_nshift['Address'] == street)&(df_nshift['Pickup date']>dt.datetime(2022,10,12))&(df_nshift['Pickup date']<dt.datetime(2022,10,14))][['Pickup date', 'DC name', 'Client name', 'Address', 'Zip', 'City','Country code', 'Carrier name', 'Sender weight (kg)']]

> Not the same Weight